## 1. Preprocessing

### 1.1 중국어 전처리 패키지

In [2]:
# 중국어 전처리 패키지
from snownlp import SnowNLP
# pd dataframe
import pandas as pd

In [3]:
s = SnowNLP(u'这个东西太贵')

In [4]:
s.words

['这个', '东西', '太', '贵']

In [5]:
for a in s.tags:
    print(a)

('这个', 'r')
('东西', 'n')
('太', 'd')
('贵', 'a')


In [6]:
s.sentiments

0.3091627951693915

### 1.2 Load data

In [22]:
def read_file(filename):
    with open(filename, "r") as file:
        lines = file.readlines()
        verb_hao = []
        for line in lines:
            verb_hao.append(line.strip().split("</B>")[1])
    return verb_hao

In [23]:
verb_hao = read_file("sample-data/verb_hao.txt")
print(len(verb_hao))
print(verb_hao[0])

10000
是约４２ｍｍ的遮光罩镜头外型尺寸为以说一尘不染，可见其密封性<U>恨好</U>。如此笔者发现独立镜头厂家适配０７２５ｘ１２２ｍｍ光学构造为１


## 2. Training

In [24]:
import torch
import numpy as np
from tqdm import tqdm

from transformers import BertTokenizer, BertModel
from transformers import BertForMaskedLM

In [26]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 2 GPU(s) available.
We will use the GPU: TITAN RTX


In [27]:
MODEL_TYPE = 'bert-base-chinese'
MAX_SIZE = 150
BATCH_SIZE = 200

In [28]:
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)
model = BertForMaskedLM.from_pretrained(MODEL_TYPE)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
input_ids = tokenizer("这个东西[MASK]贵")

In [30]:
input_ids

{'input_ids': [101, 6821, 702, 691, 6205, 103, 6586, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [33]:
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [35]:
outputs = model(input_ids['input_ids'],input_ids['attention_mask'],input_ids['token_type_ids'])

AttributeError: 'list' object has no attribute 'size'

In [14]:
torch.argmax(outputs[1][0][5])

tensor(1922)

In [15]:
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [16]:
with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)

In [17]:
logits = outputs[0].numpy()